### Local에서의 PySpark 환경 설정

In [1]:
import os
os.environ['PYSPARK_PYTHON'] = "C:/Users/user/anaconda3/envs/torchenv/python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = "C:/Users/user/anaconda3/envs/torchenv/python.exe"

### SparkSession 생성

In [41]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructField, StructType, IntegerType, StringType

spark = SparkSession.builder\
                    .appName('mostpopularSuperhero')\
                    .getOrCreate() # 이미 존재하면 그 DataFrame 사용, 없다면 새롭게 생성 
spark

### Schema 정의

In [42]:
schema = StructType([
  # id 필드를 정의, 데이터 타입은 Integer
  StructField('id', IntegerType(), True),
  StructField('name', StringType(), True)
])

schema

StructType([StructField('id', IntegerType(), True), StructField('name', StringType(), True)])

In [43]:
names = spark.read.schema(schema).option('sep',' ').csv('file:///nvidia_course/nvidia-course/week1/day5/AdvancedSpark/Marvel+Names')
names.show(10)

+---+--------------------+
| id|                name|
+---+--------------------+
|  1|24-HOUR MAN/EMMANUEL|
|  2|3-D MAN/CHARLES CHAN|
|  3|    4-D MAN/MERCURIO|
|  4|             8-BALL/|
|  5|                   A|
|  6|               A'YIN|
|  7|        ABBOTT, JACK|
|  8|             ABCISSA|
|  9|                ABEL|
| 10|ABOMINATION/EMIL BLO|
+---+--------------------+
only showing top 10 rows



In [44]:
lines = spark.read.text("file:///nvidia_course/nvidia-course/week1/day5/AdvancedSpark/Marvel+Graph")
lines.show(10), type(lines)

+--------------------+
|               value|
+--------------------+
|5988 748 1722 375...|
|5989 4080 4264 44...|
|5982 217 595 1194...|
|5983 1165 3836 43...|
|5980 2731 3712 15...|
|5981 3569 5353 40...|
|5986 2658 3712 26...|
|5987 2614 5716 17...|
|5984 590 4898 745...|
|5985 3233 2254 21...|
+--------------------+
only showing top 10 rows



(None, pyspark.sql.dataframe.DataFrame)

In [45]:
# lines의 value 값을 공백으로 분리한 후에 첫 번째 인덱스 값을 id로 새롭게 컬럼을 생성 
# id의 값에 size - 1(자기 자신을 제외)을 계산해서 connections 컬럼 생성 
# 결국 한 id에 대해 해당 리스트에 존재하는 다른 id와는 모두 connection 되어 있는 상태 
connections = lines.withColumn('id', F.split(F.col('value')," ")[0])\
                   .withColumn('connections', F.size(F.split(F.col('value')," ")) - 1)\
                   .groupby('id').agg(F.sum('connections').alias('connections'))
                   
connections.show(10)

+----+-----------+
|  id|connections|
+----+-----------+
| 691|          7|
|1159|         12|
|3959|        143|
|1572|         36|
|2294|         15|
|1090|          5|
|3606|        172|
|3414|          8|
| 296|         18|
|4821|         17|
+----+-----------+
only showing top 10 rows



### connections이 가장 많은 노드의 id

In [46]:
most_popular = connections.sort(F.desc(F.col('connections'))).first()
most_popular

Row(id='859', connections=1937)

### connections이 가장 많은 노드의 이름

In [47]:
most_popular[0] # 슬라이싱으로 접근 가능

'859'

In [48]:
most_popular_name = names.filter(F.col('id') == most_popular[0]).select('name').first()
most_popular_name[0]

'CAPTAIN AMERICA'

In [49]:
print(f"{most_popular_name[0]} is the most popular superhero with {str(most_popular[1])}")


CAPTAIN AMERICA is the most popular superhero with 1937


### connections이 가장 적은 노드의 id, 이름

In [50]:
worst_popular = connections.sort(F.asc(F.col('connections'))).first()
worst_popular

Row(id='467', connections=1)

In [51]:
worst_popular_name = names.filter(F.col('id') == worst_popular[0]).select('name').first()
worst_popular_name[0]

'BERSERKER II'

### connections dataframe에 name 컬럼을 추가한 후 `orderby()`로 정렬하려 추출

In [52]:
name_with_connections = connections.join(names, on = 'id')
name_with_connections

DataFrame[id: string, connections: bigint, name: string]

In [53]:
worst_popular_row = name_with_connections.orderBy(func.asc('connections')).first()
print(f"{worst_popular_row['name']} is the most popular superhero with {worst_popular_row['connections']}")


BERSERKER II is the most popular superhero with 1


In [ ]:
# Spark 세션은 항상 작업 종료 후에 종료
spark.stop()